# 第3章: 役割の割り当て（ロールプロンプティング）

- [レッスン](#lesson)
- [演習](#exercises)
- [サンプル実験場](#example-playground)

## セットアップ

以下のセットアップセルを実行して、APIキーを読み込み、`get_completion`ヘルパー関数を確立してください。

In [ ]:
# Import python's built-in regular expression library
import re
import boto3
import json

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Retrieve the MODEL_NAME variable from the IPython store
%store -r MODEL_NAME
%store -r AWS_REGION

client = boto3.client('bedrock-runtime',region_name=AWS_REGION)

def get_completion(prompt,system=''):
    body = json.dumps(
        {
            "anthropic_version": '',
            "max_tokens": 2000,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.0,
            "top_p": 1,
            "system": system
        }
    )
    response = client.invoke_model(body=body, modelId=MODEL_NAME)
    response_body = json.loads(response.get('body').read())

    return response_body.get('content')[0].get('text')

---

## レッスン

Claudeがあなたの言葉以外にコンテキストを持たないというテーマの続きで、時には**Claudeに特定の役割を担わせる（必要なコンテキストをすべて含めて）ことが重要**です。これはロールプロンプティングとも呼ばれます。役割のコンテキストの詳細が多いほど、より良い結果が得られます。

**Claudeに役割を与えることで、書く、コーディング、要約など、様々な分野でClaudeのパフォーマンスを向上させることができます。** これは、人間が「______のように考えて」と言われると時々助けになることと似ています。ロールプロンプティングは、Claudeの回答のスタイル、トーン、態度も変えることができます。

**注意:** ロールプロンプティングは、システムプロンプトまたはUserメッセージターンの一部として行うことができます。

### 例

以下の例では、ロールプロンプティングなしでは、Claudeがスケートボードについて一文での視点を求められた時に**簡潔で装飾のない答え**を提供していることがわかります。

しかし、Claudeに猫の役割を演じるよう指示すると、Claudeの視点が変わり、その結果**Claudeの回答のトーン、スタイル、内容が新しい役割に適応します**。

**注意:** 使用できるボーナステクニックとして、**Claudeに意図した聞き手についてのコンテキストを提供する**ことがあります。以下では、Claudeが誰に話しかけるべきかも伝えるためにプロンプトを調整することもできたでしょう。「あなたは猫です」と「あなたはスケートボーダーの群衆に話しかける猫です」では、まったく異なる回答が得られます。

こちらがシステムプロンプトでのロールプロンプティングなしのプロンプトです：

In [ ]:
# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print Claude's response
print(get_completion(PROMPT))

こちらは同じユーザーの質問ですが、ロールプロンプティングありです。

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a cat."

# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

ロールプロンプティングは、Claudeに特定のスタイルで文章を書かせたり、特定の声で話させたり、回答の複雑さを導くための方法として使用できます。**ロールプロンプティングは、Claudeを数学や論理タスクの実行においてもより優秀にすることができます。**

例えば、以下の例では明確な正解があり、それは「はい」です。しかし、Claudeは間違って情報が不足していると考えており、実際にはそうではありません：

In [ ]:
# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Claude's response
print(get_completion(PROMPT))

では、**Claudeに論理ボットとして行動するよう指示したら**どうでしょうか？これによってClaudeの回答はどう変わるでしょうか？

この新しい役割の割り当てにより、Claudeは正解を導き出します。（ただし、必ずしもすべて正しい理由でそうしているわけではありません）

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a logic bot designed to answer complex logic problems."

# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

**注意:** このコースを通じて学ぶことは、**似たような結果を得るために使用できるプロンプトエンジニアリング技術がたくさんある**ということです。どの技術を使うかはあなた次第です！あなた自身の**プロンプトエンジニアリングスタイルを見つけるために実験することをお勧めします**。

上記の内容を変更せずにレッスンプロンプトを試したい場合は、レッスンnotebookの一番下までスクロールして[**サンプル実験場**](#example-playground)をご覧ください。

---

## 演習
- [演習 3.1 - 数学の訂正](#exercise-31---math-correction)

### 演習 3.1 - 数学の訂正
一部の場面では、**Claudeは数学、単純な数学でも苦労することがあります**。以下では、Claudeは数学問題を正しく解かれていると誤って評価しており、第2ステップに明らかな計算ミスがあるにもかかわらずです。Claudeは実際にステップバイステップで検証する際にはミスを捉えているのに、全体の解答が間違っているという結論に飛ばないことに注意してください。

`PROMPT`および/または`SYSTEM_PROMPT`を修正して、Claudeが解答を正しく解かれているのではなく`incorrectly`（間違って）解かれていると評価するようにしてください。

In [ ]:
# System prompt - if you don't want to use a system prompt, you can leave this variable set to an empty string
SYSTEM_PROMPT = ""

# Prompt
PROMPT = """Is this equation solved correctly below?

2x - 3 = 9
2x = 6
x = 3"""

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    if "incorrect" in text or "not correct" in text.lower():
        return True
    else:
        return False

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが欲しい場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_3_1_hint)

### おめでとうございます！

ここまでのすべての演習を解くことができたら、次の章に進む準備ができています。楽しいプロンプト作成を！

---

## サンプル実験場

ここは、このレッスンで示されたプロンプトの例を自由に実験し、プロンプトを調整してClaudeの応答にどのような影響があるかを確認する場所です。

In [ ]:
# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a cat."

# Prompt
PROMPT = "In one sentence, what do you think about skateboarding?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a logic bot designed to answer complex logic problems."

# Prompt
PROMPT = "Jack is looking at Anne. Anne is looking at George. Jack is married, George is not, and we don’t know if Anne is married. Is a married person looking at an unmarried person?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))